In [1]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import pyvips
import cv2

C:\Users\Marina\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv(r'D:\4th year\Deep Learning\project\train.csv')
image_dir = r'D:\4th year\Deep Learning\project\train_thumbnails'
masked_images_dir = r'D:\4th year\Deep Learning\project\masked_images'

In [3]:
label_mapping = {
    "CC": 0,
    "EC": 1,
    "HGSC": 2,
    "LGSC": 3,
    "MC": 4,
}


In [5]:
preprocessed_data = []

for filename in os.listdir(masked_images_dir):
        
        # extract image id
        img_path = os.path.join(masked_images_dir, filename)
        image_id = filename.split('_')[1].split('.')[0]
        
        # extract label
        label = data[data['image_id'] == int(image_id)]['label'].values  
        label = label[0]  
        numerical_label = label_mapping[label]
        one_hot_label = np.zeros(len(label_mapping))
        one_hot_label[numerical_label] = 1

        img = Image.open(img_path)
        # resize
        resized_img = img.resize((512, 512), Image.ANTIALIAS)
        img = np.array(resized_img)
        # Convert to grayscale
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        # Apply Otsu's thresholding (black and white pixels)
        _, thresholded = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        # Apply the mask to the original image
        masked_img = cv2.bitwise_and(img, img, mask=thresholded)

        img_array = np.array(masked_img)

        preprocessed_data.append((img_array, one_hot_label))



C:\Users\Marina\AppData\Local\Temp\ipykernel_20628\3295317964.py:17: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  resized_img = img.resize((512, 512), Image.ANTIALIAS)


In [6]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(preprocessed_data, test_size=0.2, random_state=42)


# Extract features and labels from the train_data and val_data
train_images, train_labels = zip(*train_data)
val_images, val_labels = zip(*val_data)

# Convert lists to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
val_images = np.array(val_images)
val_labels = np.array(val_labels)

In [7]:
num_classes = 5
batch_size = 8

# the model
base_model = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet', input_shape=(512, 512, 3)
)

flatten = tf.keras.layers.Flatten()
fc1 = tf.keras.layers.Dense(num_classes, activation='softmax')

inputs = tf.keras.Input(shape=(512, 512, 3))
x = base_model(inputs)
x = flatten(x)
outputs = fc1(x)


model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    train_images, train_labels,
    batch_size=batch_size,
    epochs=20,
    validation_data=(val_images, val_labels)
)

Epoch 1/20
16/16 [==============================] - 133s 7s/step - loss: 7.7963 - accuracy: 0.3140 - val_loss: 4.5900 - val_accuracy: 0.3871
Epoch 2/20
16/16 [==============================] - 118s 7s/step - loss: 0.5117 - accuracy: 0.9256 - val_loss: 11.3027 - val_accuracy: 0.3226
Epoch 3/20
16/16 [==============================] - 121s 8s/step - loss: 0.0752 - accuracy: 0.9669 - val_loss: 10.3389 - val_accuracy: 0.2903
Epoch 4/20
 4/16 [======>.......................] - ETA: 1:31 - loss: 4.3418e-04 - accuracy: 1.0000

KeyboardInterrupt: 